In [2]:
import rospy
import actionlib
import PyKDL
import tf2_ros

from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import Twist, Pose, Pose2D, PoseStamped
from nav_msgs.msg import Odometry, Path
from gazebo_msgs.msg import ModelState, ContactsState

from threading import Event
import copy
import trimesh
import numpy as np

from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2

from roomor.generator import CubeRoomGenerator

In [7]:
class RosBridge:
    
    def __init__(self, real_robot=False):
        
        # Event is clear while initialization or set_state is going on
        self.reset = Event()
        self.reset.clear()
        self.get_state_event()
        self.get_state_event.set()
        
        self.real_robot = real_robot
        
        self.move_base_client = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
        self.mir_exec_path = rospy.Publisher('mir_exec_path', Path, queue_size=10)
        
        if self.real_robot:
            rospy.Subscriber("odom", Odometry, self.callbackOdometry, queue_size=1)
        else:
            rospy.Subscriber("odom_comb", Odometry, self.callbackOdometry, queue_size=1)
            
        rospy.Subscriber("mir_collision", ContactsState, self.collision_callback)
        
        self.mir_pose = [0.0] * 3
        self.mir_twist = [0.0] * 2
        self.collision = False
        
        self.room_generator = None
        self.room_generator_params = dict()
        
        # Room generator parameter
        self.room_generator_params['obstacle_count'] = 10
        self.room_generator_params['obstacle_size'] = 0.7
        self.room_generator_params['target_size'] = 0.2
        self.room_generator_params['room_length_max'] = 9
        self.room_generator_params['room_mass_min'] = 20
        self.room_generator_params['room_mass_max'] = 36
        self.room_generator_params['room_wall_height'] = 0.8
        self.room_generator_params['room_wall_thickness'] = 0.05
        
        # parameter as not state
        self.room_generator_params['agent_size'] = 0.3
        self.room_generator_params['wall_threshold'] = 0.1
        
        # Room's infomation
        self.obstacles = [] # [3,n]
        self.targets = [] # [3,n]
        
        self.path_frame = 'map'
        
        if self.real_robot:
            self.path_frame = 'world'

            # Apply transform to center the robot, with real_robot we use World frame,
            # World is the Map frame translated in XY to center robot
            tfBuffer = tf2_ros.Buffer()
            listener = tf2_ros.TransformListener(tfBuffer)

            trans = tfBuffer.lookup_transform("world", "map", rospy.Time(), rospy.Duration(1.0))
            v = PyKDL.Vector(trans.transform.translation.x,trans.transform.translation.y,trans.transform.translation.z)
            r = PyKDL.Rotation.Quaternion(trans.transform.rotation.x,trans.transform.rotation.y,trans.transform.rotation.z,trans.transform.rotation.w)
            self.world_to_map = PyKDL.Frame(r,v)
        
        rospy.Subscriber("robot_pose", Pose, self.callbackState, queue_size=1)
        
        # Initialize Path
        self.mir_path = Path()
        self.mir_path.header.stamp = rospy.Time.now()
        self.mir_path.header.frame_id = self.path_frame

        # Flag indicating if it is safe to move backwards
        self.safe_to_move_back = True
        # Flag indicating if it is safe to move forward
        self.safe_to_move_front = True
        self.rate = rospy.Rate(10) #30Hz
        self.reset.set()
        
    def get_state(self):
        self.get_state_event.clear()
        # Get environment state
        state = []
        mir_pose = copy.deepcopy(self.mir_pose)
        mir_twist = copy.deepcopy(self.mir_twist)
        is_collision = copy.deepcopy(self.collision)
        
        obstacle_count = 0
        obstacle_size = 0.0
        target_size = 0.0
        room_length_max = 0.0
        room_mass_min = 0.0
        room_mass_max = 0.0
        room_wall_height = 0.0
        room_wall_thickness = 0.0

        obstacles = copy.deepcopy(self.obstacles)
        targets = copy.deepcopy(self.targets)
        
        self.get_state_event.set()
        
        # Create and fill State message
        msg = robot_server_pb2.State()
        msg.state.extend(mir_pose)
        msg.state.extend(mir_twist)
        msg.state.extend([is_collision])
        msg.state.extend([obstacle_count])
        msg.state.extend([obstacle_size])
        msg.state.extend([target_size])
        msg.state.extend([room_length_max])
        msg.state.extend([room_mass_min])
        msg.state.extend([room_mass_max])
        msg.state.extend([room_wall_height])
        msg.state.extend([room_wall_thickness])
        msg.state.extend(obstacles)
        msg.state.extend(targets)
        msg.success = 1
        
        return msg
        
    def set_state(self, state_msg):
        state = state_msg.state
        
        self.reset.clear()
        
        self.mir_path=Path()
        self.mir_path.header.stamp=rospy.Time.now()
        self.mir_path.header.frame_id= self.path_frame
        
        self.room_generator_params['obstacle_count'] = copy.deepcopy(state[6])
        self.room_generator_params['obstacle_size'] = copy.deepcopy(state[7])
        self.room_generator_params['target_size'] = copy.deepcopy(state[8])
        self.room_generator_params['room_length_max'] = copy.deepcopy(state[9])
        self.room_generator_params['room_mass_min'] = copy.deepcopy(state[10])
        self.room_generator_params['room_mass_max'] = copy.deepcopy(state[11])
        self.room_generator_params['room_wall_height'] = copy.deepcopy(state[12])
        self.room_generator_params['room_wall_thickness'] = copy.deepcopy(state[13])
        
        if not self.real_robot:
            if self.room_generator is None:
                room = self.gen_simulation_room(new_generator=True)
            else:
                room = self.gen_simulation_room(new_generator=False)

            # Spawn models to Gazebo world and sleep until it done
            room.spawn_all()
        
        self.reset.set()
        
        return 1
        
            
    def gen_simulation_room(self, generator_params, new_generator=False):
        if new_generator:
            self.room_generator = CubeRoomGenerator(**generator_param)
        
        return self.room_generator.generate_new()
    
    def gen_agent_state(self, room, threshold):
        freezone = room.get_freezone_poly().buffer(-1*threshold, cap_style=2, join_style=2)
        pos_x, pos_y = trimesh.path.polygons.sample(freezone, 1)
        ori_z = np.random.rand()*np.pi*2
        
        start_state = ModelState()
        start_state.model_name = 'mir'
        start_state.pose.position.x = pos_x
        start_state.pose.position.y = pos_y
        orientation = PyKDL.Rotation.RPY(0,0, ori_z)
        start_state.pose.orientation.x, start_state.pose.orientation.y, start_state.pose.orientation.z, start_state.pose.orientation.w = orientation.GetQuaternion()
        
        start_state.twist.linear.x = 0.0
        start_state.twist.linear.y = 0.0
        start_state.twist.linear.z = 0.0
        start_state.twist.angular.x = 0.0
        start_state.twist.angular.y = 0.0
        start_state.twist.angular.z = 0.0
        
        return start_state
    
    def publish_env_cmd_vel(self, lin_vel, ang_vel):
        if (not self.safe_to_move_back) or (not self.safe_to_move_front):
            # If it is not safe to move overwrite velocities and stop robot
            rospy.sleep(0.07)
            return 0.0, 0.0
        msg = Twist()
        msg.linear.x = lin_vel
        msg.angular.z = ang_vel
        self.env_cmd_vel_pub.publish(msg)
        # Sleep time set manually to achieve approximately 10Hz rate
        rospy.sleep(0.07)
        return lin_vel, ang_vel
    
    def odometry_callback(self, data):
        # Save robot velocities from Odometry internally
        self.robot_twist=data.twist.twist
        